# Install and Import Dependencies

In [ ]:
# tensorflow
# opencv-python --> Leverage the webcam in real time to be able to perform pose estimation

!pip install tensorflow==2.4.1 opencv-python

In [1]:
import tensorflow as tf # CORE DEPENDENCY FOR THE PROJECT TO USE MOVENET LIGHTHNING WHICH IS THE FASTEST VERSION
import numpy as np # REQUIRED FOR PERFORMING AND WRITING RENDERING FUNCTIONS
import cv2  # CAPTURE FEED FROM WEBCAM

C:\Users\varun\anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\varun\anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
C:\Users\varun\anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


# Load Model

In [2]:
# DOWNLOAD THE MODEL FROM https://tfhub.dev/google/lite-model/movenet/singlepose/lightning/3

interpreter = tf.lite.Interpreter(model_path='lite-model_movenet_singlepose_lightning_3.tflite')
interpreter.allocate_tensors() # LEVERAGING THE MODEL

# Make Detections

In [6]:
img = frame.copy()

In [7]:
img.shape # (height, width, deep)

(480, 640, 3)

In [9]:
interpreter.get_input_details()
#interpreter.get_output_details()

[{'name': 'serving_default_input:0',
  'index': 0,
  'shape': array([  1, 192, 192,   3]),
  'shape_signature': array([  1, 192, 192,   3]),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [36]:
cap = cv2.VideoCapture(0) # CONNECTING TO WEBCAM
while cap.isOpened():
    ret, frame = cap.read() # READ THE FRAME FROM WEBCAM AND UNPACK THE RESULTS
    
    # Reshape image
    img = frame.copy()
    img = tf.image.resize_with_pad(np.expand_dims(img, axis=0), 192,192) 
    # np.expand_dims(img, axis=0) this line is encapsulating in the shape of (1, 480, 640, 3) and we need new dimension because
    # Inputs: A frame of video or an image, represented as an float32 tensor of shape: 192x192x3. Channels order: RGB with values in [0, 255].(Ref: Tensorflow Hub)
    input_image = tf.cast(img, dtype=tf.float32)
    
    # Setup input and output 
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    
    # Make predictions 
    interpreter.set_tensor(input_details[0]['index'], np.array(input_image)) # above line output # it is the reshaped input_image
    interpreter.invoke()
    keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])
    print(keypoints_with_scores)
    
    # Rendering q
    draw_connections(frame, keypoints_with_scores, EDGES, 0.4)
    draw_keypoints(frame, keypoints_with_scores, 0.4)
    q
    
    cv2.imshow('MoveNet Lightning', frame)
    
    if cv2.waitKey(10) & 0xFF==ord('q'): # CHECKING WHETHER WE HIT THE KEY Q IN THE FRAME
        break
        
cap.release()
cv2.destroyAllWindows()

[[[[0.3425042  0.22065583 0.72945964]
   [0.28317112 0.26840284 0.6317643 ]
   [0.29474568 0.18931787 0.58435464]
   [0.26794124 0.366361   0.596121  ]
   [0.2988808  0.19571064 0.37924916]
   [0.34723336 0.50275946 0.71931905]
   [0.4519652  0.1873406  0.8329585 ]
   [0.7389595  0.51477814 0.08749765]
   [0.7236566  0.05967354 0.69937444]
   [0.7581713  0.35501784 0.03698114]
   [0.75984764 0.01885112 0.05278483]
   [0.8135299  0.51549494 0.5342757 ]
   [0.8108226  0.33469936 0.68176997]
   [0.8470025  0.44143352 0.02237606]
   [0.85074556 0.03320462 0.04163253]
   [0.85665977 0.41859245 0.01419291]
   [0.87180465 0.22596279 0.01043576]]]]
[[[[0.34131378 0.21443525 0.6736468 ]
   [0.2806268  0.26092762 0.743541  ]
   [0.29107141 0.18515413 0.5799088 ]
   [0.263183   0.36284614 0.6531394 ]
   [0.29493374 0.19246243 0.43340194]
   [0.3473235  0.50507164 0.74128455]
   [0.45481238 0.17962581 0.76799756]
   [0.7334266  0.51544905 0.09753332]
   [0.7293068  0.05703206 0.67347044]
   [0.757

In [12]:
keypoints_with_scores.shape

#Outputs: A float32 tensor of shape [1, 1, 17, 3].

(1, 1, 17, 3)

In [15]:
# 17 keypoints (in the order of: [nose, left eye, right eye, left ear, right ear, left shoulder, right shoulder, left elbow, right elbow, left wrist, right wrist, left hip, right hip, left knee, right knee, left ankle, right ankle]

#right_ear = keypoints_with_scores[0][0][4]
left_hip = keypoints_with_scores[0][0][11]
left_hip

array([1.002311 , 0.7262043, 0.0155628], dtype=float32)

In [18]:
interpreter.get_output_details()[0]['index']

312

In [20]:
interpreter.get_tensor(interpreter.get_output_details()[0]['index']) # getting our results out

array([[[[0.54236895, 0.5320836 , 0.62488604],
         [0.47426495, 0.5884714 , 0.61957145],
         [0.47567102, 0.4848126 , 0.5695413 ],
         [0.4357023 , 0.66381   , 0.7312067 ],
         [0.4439237 , 0.42843422, 0.60033834],
         [0.6479767 , 0.81146663, 0.7576417 ],
         [0.65359986, 0.3275717 , 0.58053356],
         [0.88291824, 0.9035157 , 0.17523658],
         [0.86396706, 0.2825033 , 0.0918197 ],
         [0.86859363, 0.9134592 , 0.04624933],
         [0.8530125 , 0.36638176, 0.01425904],
         [1.002311  , 0.7262043 , 0.0155628 ],
         [0.8929889 , 0.9681789 , 0.02610645],
         [0.7341444 , 0.9671969 , 0.03937012],
         [0.7151288 , 0.0143925 , 0.02306315],
         [0.738474  , 0.98740613, 0.01244113],
         [0.72171444, 0.01508609, 0.02526969]]]], dtype=float32)

In [21]:
shaped = np.squeeze(np.multiply(interpreter.get_tensor(interpreter.get_output_details()[0]['index']), [480,640,1]))

In [24]:
# extracting coordinates
for kp in shaped:
    ky, kx, kp_conf = kp
    print(int(ky), int(kx), kp_conf)

260 340 0.6248860359191895
227 376 0.6195714473724365
228 310 0.5695412755012512
209 424 0.7312067151069641
213 274 0.600338339805603
311 519 0.7576416730880737
313 209 0.5805335640907288
423 578 0.17523658275604248
414 180 0.09181970357894897
416 584 0.046249330043792725
409 234 0.014259040355682373
481 464 0.01556280255317688
428 619 0.026106446981430054
352 619 0.03937011957168579
343 9 0.02306315302848816
354 631 0.012441128492355347
346 9 0.025269687175750732


# Draw Keypoints

In [28]:
def draw_keypoints(frame, keypoints, confidence_threshold): # image, keypoints, confidence_threshold
    y, x, c = frame.shape # YX and channel cordinates
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 4, (255,0,0), -1) # BGR

# Draw Edges

In [31]:
# Different body connections
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

In [33]:
shaped[0], shaped[1]


(array([260.33709526, 340.53348541,   0.62488604]),
 array([227.64717579, 376.6217041 ,   0.61957145]))

In [32]:
for edge, color in EDGES.items():
    p1, p2 = edge
    y1, x1, c1 = shaped[p1]
    y2, x2, c2 = shaped[p2]
    print((int(x2), int(y2)))

(376, 227)
(310, 228)
(424, 209)
(274, 213)
(519, 311)
(209, 313)
(578, 423)
(584, 416)
(180, 414)
(234, 409)
(209, 313)
(464, 481)
(619, 428)
(619, 428)
(619, 352)
(631, 354)
(9, 343)
(9, 346)


In [34]:
def draw_connections(frame, keypoints, edges, confidence_threshold): 
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 2)